# File Name: simple_knwl_bases_retrival.ipynb
### Location: Chapter 7
### Purpose: 
#####             1. Use the RetrieveAndGenerate API for Amazon Bedrock integration. 
##### Dependency: simple_knwl_bases_building.ipynb at Chapter 7 should executed properly. 
# <ins>-----------------------------------------------------------------------------------</ins>

# <ins>Amazon SageMaker Classic</ins>
#### Those who are new to Amazon SageMaker Classic. Follow the link for the details. https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html

# <ins>Environment setup of Kernel</ins>
##### Fill "Image" as "Data Science"
##### Fill "Kernel" as "Python 3"
##### Fill "Instance type" as "ml-t3-medium"
##### Fill "Start-up script" as "No Scripts"
##### Click "Select"

###### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-create-open.html for details.

# <ins>Mandatory installation on the kernel through pip</ins>

##### This lab will work with below software version. But, if you are trying with latest version of boto3, awscli, and botocore. This code may fail. You might need to change the corresponding api. 

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell. 

In [ ]:
%pip install --no-build-isolation --force-reinstall -q \
    "boto3>=1.34.84" \
    "opensearch-py>=2.7.1" \
    "retrying>=1.3.4" \
    "ragas" \
    "ipywidgets>=7.6.5" \
    "iprogress>=0.4" \
    "langchain>=0.2.16" \
    "langchain_community>=0.2.17" \
    "awscli>=1.32.84" \
    "botocore>=1.34.84" \
    "langchain-aws>=0.1.7"    

# <ins>Disclaimer</ins>

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell.

# <ins>Restart the kernel</ins>

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# <ins>Python package import</ins>

##### boto3 offers various clients for Amazon Bedrock to execute various actions.
##### botocore is a low-level interface to AWS tools, while boto3 is built on top of botocore and provides additional features

In [ ]:
import json
import os
import boto3
import botocore
import pprint
import random
from retrying import retry
import warnings
import time
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, RequestError
from botocore.exceptions import NoCredentialsError, PartialCredentialsError
import pprint as pp
from botocore.exceptions import BotoCoreError, ClientError

### Ignore warning 

In [ ]:
warnings.filterwarnings('ignore')

### %store magic command to retrive all the variable value from other notebook. 
### Here, simple_knwl_bases_building.ipynb

In [ ]:
%store -r 

## Define important environment variable

In [ ]:
%%time

# Try-except block to handle potential errors
try:
    # Create a new Boto3 session to interact with AWS services
    boto3_session_name = boto3.session.Session()

    # Create a Bedrock Agent client using the current session and region
    bedrock_agent_client = boto3_session_name.client('bedrock-agent', region_name=aws_region_name)

    # Create an S3 client to interact with Amazon S3
    s3_client = boto3.client('s3')

    # Create an STS client to interact with AWS Security Token Service (STS)
    sts_client = boto3.client('sts')


    # Create an OpenSearch Serverless (AOSS) client using the current session
    aoss_client = boto3_session_name.client('opensearchserverless')


    # Create an IAM client to interact with Identity and Access Management (IAM) service
    iam_client = boto3_session_name.client('iam')

    # Retrieve the current AWS account number and ARN of the caller
    sts_client = boto3.client('sts')
    identity_arn = sts_client.get_caller_identity().get('Arn')
    
    # boto3_bedrock_agent_runtime_client
    boto3_bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region_name=aws_region_name)
    
    # Store all variables in a dictionary
    variables_store = {
        "aws_region_name": aws_region_name,
        "bedrock_agent_client": bedrock_agent_client,
        "opensearch_service_name": opensearch_service_name,
        "s3_client": s3_client,
        "sts_client": sts_client,
        "aws_account_id": aws_account_id,
        "s3_suffix": s3_suffix,
        "s3_bucket_name": s3_bucket_name,
        "aoss_client": aoss_client,
        "vector_store_name": vector_store_name,
        "index_name": index_name,
        "iam_client": iam_client,
        "sts_client": sts_client,
        "identity_arn": identity_arn,
        "bedrock_knowledge_bases_name": bedrock_knowledge_bases_name,
        "aoss_collectionarn": aoss_collectionarn,
        "aoss_collection_host": aoss_collection_host,
        "genaibookedbedrocksagemakerexecutionrolearn": genaibookedbedrocksagemakerexecutionrolearn,
        "knowledgeBaseId": knowledgeBaseId
    }

    # Print all variables
    for var_name, value in variables_store.items():
        print(f"{var_name}: {value}")

except Exception as e:
    print(f"An unexpected error occurred: {e}")


#### Using RetrieveAndGenerate API 
#### Refer: https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_RetrieveAndGenerate.html 
####        https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html#
###### Purpose: Queries a knowledge base and generates responses based on the retrieved results and using the specified foundation model or inference profile. 
######          The response only cites sources that are relevant to the query.

In [ ]:
%%time

# Function to query the knowledge base and generate a response
def ask_bedrock_llm_with_knowledge_base(client, query: str, model_arn: str, kb_id: str) -> dict:
    try:
        response = client.retrieve_and_generate(
            input={'text': query},
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kb_id,
                    'modelArn': model_arn
                }
            },
        )
        return response
    except (BotoCoreError, ClientError) as e:
        print(f"Error during API call to retrieve and generate response: {str(e)}")
        raise

# Function to process and print the generated response and its citations
def process_and_print_response(model_name: str, response: dict):
    try:
        generated_text = response['output']['text']
        citations = response.get("citations", [])
        contexts = []

        for citation in citations:
            retrieved_references = citation.get("retrievedReferences", [])
            for reference in retrieved_references:
                contexts.append(reference["content"]["text"])

        print(f"---------- Generated using {model_name}:")
        print(generated_text)
        print()
        print()
        print(f"---------- Citations for the response generated by {model_name}:")
        print(contexts)
        print()

    except KeyError as e:
        print(f"Key error while processing response: {str(e)}")
        raise

# Main execution section
prompt = "What is Amazon doing and cashflow?"

# List of Bedrock models with names and model codes
bedrock_model_ids = [
        ["Anthropic Claude Haiku", "anthropic.claude-3-haiku-20240307-v1:0"]
    ]

# Initialize Bedrock client
try:

# Loop through each Claude model to generate and print results
    for model_id in bedrock_model_ids:
        model_name, model_code = model_id
        model_arn = f'arn:aws:bedrock:{aws_region_name}::foundation-model/{model_code}'
            
        try:
            # Query the knowledge base with the specified model
            response = ask_bedrock_llm_with_knowledge_base(boto3_bedrock_agent_runtime_client, prompt, model_arn, knowledgeBaseId)
            # Process and display the generated results
            process_and_print_response(model_name, response)
            
        except Exception as e:
                print(f"An error occurred with model {model_name}: {str(e)}")

except Exception as e:
    print(f"An error occurred while initializing the client or querying models: {str(e)}")

# End of NoteBook 

#### <ins>Step 1</ins> 

##### Please ensure that you close the kernel after using this notebook to avoid any potential charges to your account.

##### Process: Go to "Kernel" at top option. Choose "Shut Down Kernel". 
##### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/studio-ui.html


#### <ins>Step 2</ins> 

#### If you are not executing any further lab of this Chapter 7
##### Execute the simple_knwl_bases_clean_up.ipynb to delete all the instances to avoid any potential charges to your account.